In [4]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [14]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [136]:
period = '_JanFeb2020'  # _JanFeb2020, _MarchApril2021, _SeptOct2020

In [137]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [138]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  4876
USA:  7519
China&USA:  12395


In [139]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 4876
1000 / 4876
2000 / 4876
3000 / 4876
4000 / 4876
0 / 7519
1000 / 7519
2000 / 7519
3000 / 7519
4000 / 7519
5000 / 7519
6000 / 7519
7000 / 7519
0 / 12395
1000 / 12395
2000 / 12395
3000 / 12395
4000 / 12395
5000 / 12395
6000 / 12395
7000 / 12395
8000 / 12395
9000 / 12395
10000 / 12395
11000 / 12395
12000 / 12395


In [140]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['talk university challenge china face deal misconception people medium watch', 'spring always bring along lively saturate sea colorful bloom flower look collection flower blossom across china enjoy upcoming new season', 'mean carnival brazil year one samba school take inspiration china', 'china world make great stride track infectious disease take unexpected turn make hard track', 'fake news story case covid lead protest evacuation people', 'transmission outside china graphic', 'world health organization announce lead team international expert currently china', 'outside china confirm case novel covid half cruise ship dock japan rest scatter among country mostly asia', 'five cargo aircraft airway head city beijing shanghai carry ton medical supply gratis include million mask hand send epidemic prevention supply purchase china government', 'case miss jinping chair lead body meeting covid control']

USA:
['shameful attack bus carry evacuee china coronavirus outbreak', 'china hubei

In [143]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  3617
USA:  3380
China&USA:  4883


In [144]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['china', 'novel', 'hospital', 'coronavirus', 'case', 'outbreak', 'new', 'patient', 'fight', 'say', 'confirm', 'people', 'epidemic', 'medical', 'hubei', 'province', 'battle', 'support', 'report', 'health', 'live', 'city', 'amid', 'death', 'day', 'control', 'virus', 'year', 'take', 'effort', 'country', 'help', 'one', 'discharge', 'work']

USA
['coronavirus', 'china', 'outbreak', 'virus', 'case', 'say', 'new', 'spread', 'death', 'people', 'health', 'late', 'report', 'cruise', 'first', 'ship']

China&USA
['china', 'coronavirus', 'outbreak', 'case', 'new', 'novel', 'say', 'virus', 'hospital', 'people', 'death', 'spread', 'health', 'patient', 'confirm', 'report', 'fight', 'city', 'late']


# less frequent words

In [146]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<5]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=5]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<5]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=5]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=5]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]

Less frequent China:  2132
More frequent China:  1485
Less frequent USA:  1709
More frequent USA: 1671
Less frequent China&USA:  2513
More frequent China&USA:  2370


In [147]:
# discard less frequent words
cleaned_mostfreq_text_China = cleaned_text_China.copy()
for txt in range(len(cleaned_mostfreq_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China))
  for word in less_frequent_words_China:
    if word in cleaned_mostfreq_text_China[txt].split():
      cleaned_mostfreq_text_China[txt] = cleaned_mostfreq_text_China[txt].replace(word, '')
      cleaned_mostfreq_text_China[txt] = " ".join(cleaned_mostfreq_text_China[txt].split())

cleaned_mostfreq_text_USA = cleaned_text_USA.copy()
for txt in range(len(cleaned_mostfreq_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_USA))
  for word in less_frequent_words_USA:
    if word in cleaned_mostfreq_text_USA[txt].split():
      cleaned_mostfreq_text_USA[txt] = cleaned_mostfreq_text_USA[txt].replace(word, '')
      cleaned_mostfreq_text_USA[txt] = " ".join(cleaned_mostfreq_text_USA[txt].split())

cleaned_mostfreq_text_China_USA = cleaned_text_China_USA.copy()
for txt in range(len(cleaned_mostfreq_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China_USA))
  for word in less_frequent_words_China_USA:
    if word in cleaned_mostfreq_text_China_USA[txt].split():
      cleaned_mostfreq_text_China_USA[txt] = cleaned_mostfreq_text_China_USA[txt].replace(word, '')
      cleaned_mostfreq_text_China_USA[txt] = " ".join(cleaned_mostfreq_text_China_USA[txt].split())

0 / 4843
1000 / 4843
2000 / 4843
3000 / 4843
4000 / 4843
0 / 7519
1000 / 7519
2000 / 7519
3000 / 7519
4000 / 7519
5000 / 7519
6000 / 7519
7000 / 7519
0 / 12362
1000 / 12362
2000 / 12362
3000 / 12362
4000 / 12362
5000 / 12362
6000 / 12362
7000 / 12362
8000 / 12362
9000 / 12362
10000 / 12362
11000 / 12362
12000 / 12362


In [148]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  1492
USA:  1673
China&USA:  2375


# Build Network

In [149]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   3018
1         novel   1385
2      hospital    969
3   coronavirus    949
4          case    863
5      outbreak    846
6           new    657
7       patient    651
8         fight    583
9           say    521
10      confirm    454
11       people    426
12     epidemic    403
13      medical    378
14        hubei    369
15     province    355
16       battle    311
17      support    310
18       report    299
19       health    294
20         live    294
21         city    293
22         amid    289
23        death    288
24          day    273
25      control    266
26        virus    263
27         year    233
28         take    223
29       effort    213

USA
           Word  Count
0   coronavirus   5134
1         china   3946
2      outbreak   1404
3         virus   1133
4          case   1110
5           say    933
6           new    906
7        spread    858
8         death    734
9        people    624
10       health    603


In [150]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [151]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [152]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [153]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                         weight
(china, novel)             1024
(china, outbreak)           644
(china, coronavirus)        619
(confirm, case)             571
(patient, hospital)         560
(china, case)               527
(china, fight)              502
(novel, coronavirus)        483
(new, case)                 479
(china, new)                427
(china, say)                422
(novel, outbreak)           394
(case, report)              368
(case, death)               357
(epidemic, china)           346
(china, support)            339
(china, hospital)           334
(china, confirm)            311
(new, confirm)              296
(china, people)             281
(china, province)           272
(china, battle)             266
(china, hubei)              258
(report, new)               245
(new, death)                245
(fight, novel)              233
(makeshift, hospital)       233
(china, effort)             232
(novel, hospital)           231
(coronavirus, outbreak)     227



#Graph


In [154]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [155]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [156]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  1492
Edges:  82720
Is connected:  True

USA:
Nodes:  1673
Edges:  78390
Is connected:  True

China&USA:
Nodes:  2375
Edges:  158504
Is connected:  True


# PageRank

In [162]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [158]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [159]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [164]:
print('China: ', len(threshold(pr_China,0.0003)))
print()
print('first:')
print(threshold(pr_China,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China,0.0003).iloc[len(threshold(pr_China,0.0003))-30:])
print()
print('USA: ', len(threshold(pr_USA,0.0003)))
print()
print('first:')
print(threshold(pr_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_USA,0.0003).iloc[len(threshold(pr_USA,0.0003))-30:])
print()
print('China&USA: ', len(threshold(pr_China_USA,0.0003)))
print()
print('first:')
print(threshold(pr_China_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China_USA,0.0003).iloc[len(threshold(pr_China_USA,0.0003))-30:])

China:  645

first:
              0         1
0         china  0.052924
1         novel  0.025584
2      hospital  0.017164
3   coronavirus  0.016114
4          case  0.015881
5      outbreak  0.015444
6       patient  0.011923
7           new  0.011853
8           say  0.011254
9         fight  0.010305
10      confirm  0.009001
11       people  0.008836
12      medical  0.007752
13     province  0.007708
14     epidemic  0.007650
15        hubei  0.007484
16       health  0.006360
17         city  0.006205
18      support  0.006089
19       report  0.005872
20        death  0.005794
21       battle  0.005586
22          day  0.005476
23        virus  0.005410
24         year  0.004939
25          one  0.004876
26         amid  0.004870
27      control  0.004748
28         take  0.004553
29      country  0.004448

last:
                0         1
615          pass  0.000313
616          suit  0.000312
617           per  0.000312
618       witness  0.000312
619      standard  0.000311

# TF-IDF

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_mostfreq_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_mostfreq_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_mostfreq_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [62]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [63]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [64]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.057393
covid          0.032344
case           0.029077
novel          0.027626
coronavirus    0.027115
new            0.025339
vaccine        0.022859
hospital       0.022712
say            0.020808
outbreak       0.019470
report         0.017295
fight          0.016957
patient        0.016629
people         0.015936
confirm        0.014500
watch          0.014347
health         0.014288
country        0.014000
day            0.013577
live           0.013453
province       0.012905
first          0.012837
city           0.012793
death          0.012317
year           0.012240
world          0.011623
epidemic       0.011616
amid           0.011082
medical        0.011039
president      0.010959
dtype: float64

USA:
coronavirus    0.049972
covid          0.048809
china          0.041930
vaccine        0.031179
case           0.028555
say            0.027612
new            0.026551
report         0.017860
death          0.017719
outbreak       0.017477
virus       

# reduced graph

In [165]:
# less frequent words:
less_important_words_China = [key for key in list(threshold_reverse(pr_China,0.0003)[0])]

less_important_words_USA = [key for key in list(threshold_reverse(pr_USA,0.0003)[0])]

less_important_words_China_USA = [key for key in list(threshold_reverse(pr_China_USA,0.0003)[0])]

In [166]:
# discard less frequent words
cleaned_mostimp_text_China = cleaned_mostfreq_text_China.copy()
for txt in range(len(cleaned_mostimp_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China))
  for word in less_important_words_China:
    if word in cleaned_mostimp_text_China[txt].split():
      cleaned_mostimp_text_China[txt] = cleaned_mostimp_text_China[txt].replace(word, '')
      cleaned_mostimp_text_China[txt] = " ".join(cleaned_mostimp_text_China[txt].split())

cleaned_mostimp_text_USA = cleaned_mostfreq_text_USA.copy()
for txt in range(len(cleaned_mostimp_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_USA))
  for word in less_important_words_USA:
    if word in cleaned_mostimp_text_USA[txt].split():
      cleaned_mostimp_text_USA[txt] = cleaned_mostimp_text_USA[txt].replace(word, '')
      cleaned_mostimp_text_USA[txt] = " ".join(cleaned_mostimp_text_USA[txt].split())

cleaned_mostimp_text_China_USA = cleaned_mostfreq_text_China_USA.copy()
for txt in range(len(cleaned_mostimp_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China_USA))
  for word in less_important_words_China_USA:
    if word in cleaned_mostimp_text_China_USA[txt].split():
      cleaned_mostimp_text_China_USA[txt] = cleaned_mostimp_text_China_USA[txt].replace(word, '')
      cleaned_mostimp_text_China_USA[txt] = " ".join(cleaned_mostimp_text_China_USA[txt].split())

0 / 4843
1000 / 4843
2000 / 4843
3000 / 4843
4000 / 4843
0 / 7519
1000 / 7519
2000 / 7519
3000 / 7519
4000 / 7519
5000 / 7519
6000 / 7519
7000 / 7519
0 / 12362
1000 / 12362
2000 / 12362
3000 / 12362
4000 / 12362
5000 / 12362
6000 / 12362
7000 / 12362
8000 / 12362
9000 / 12362
10000 / 12362
11000 / 12362
12000 / 12362


In [167]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  663
USA:  611
China&USA:  652


In [168]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         china   3008
1         novel   1385
2      hospital    969
3   coronavirus    949
4          case    863
5      outbreak    846
6           new    657
7       patient    651
8         fight    583
9           say    521
10      confirm    454
11       people    426
12     epidemic    403
13      medical    378
14        hubei    369
15     province    355
16       battle    311
17      support    310
18       report    299
19         live    294
20       health    294
21         city    293
22         amid    289
23        death    288
24          day    273
25      control    266
26        virus    263
27         year    233
28         take    223
29       effort    213

USA
           Word  Count
0   coronavirus   5134
1         china   3946
2      outbreak   1404
3         virus   1133
4          case   1110
5           say    933
6           new    906
7        spread    858
8         death    734
9        people    624
10       health    603


In [169]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [170]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [171]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                         weight
(china, novel)             1018
(china, outbreak)           644
(china, coronavirus)        619
(confirm, case)             571
(patient, hospital)         560
(china, case)               527
(china, fight)              496
(novel, coronavirus)        483
(new, case)                 479
(china, new)                426
(china, say)                422
(novel, outbreak)           394
(case, report)              368
(case, death)               357
(epidemic, china)           346
(china, hospital)           334
(china, support)            333
(china, confirm)            311
(new, confirm)              296
(china, people)             278
(china, province)           272
(china, battle)             266
(china, hubei)              258
(new, death)                245
(report, new)               245
(makeshift, hospital)       233
(fight, novel)              233
(china, effort)             232
(novel, hospital)           231
(coronavirus, outbreak)     227



In [172]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [173]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [174]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  663
Edges:  46516
Is connected:  True

USA:
Nodes:  611
Edges:  40691
Is connected:  True

China&USA:
Nodes:  652
Edges:  63889
Is connected:  True


#Save edge list

In [175]:
filename = './edgelist_China_1_700.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_1_700.csv
files.download("edgelist_China_1_700.csv")

filename = './edgelist_USA_1_700.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_1_700.csv
files.download("edgelist_USA_1_700.csv")

filename = './edgelist_China_USA_1_700.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_1_700.csv
files.download("edgelist_China_USA_1_700.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [125]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [176]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_1_700")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_1_700")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_1_USA_700")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
            Id        Label
0        china        china
1        novel        novel
2     hospital     hospital
3  coronavirus  coronavirus
4         case         case

USA:
            Id        Label
0  coronavirus  coronavirus
1        china        china
2     outbreak     outbreak
3        virus        virus
4         case         case

China&USA:
            Id        Label
0        china        china
1  coronavirus  coronavirus
2     outbreak     outbreak
3         case         case
4          new          new

